### Bayesian Optimization of the Pion Lucite Detector with Integer Parameter and Constraint Bounds

This notebook demonstrates the use of Bayesian optimization to optimize the pion lucite detector design for the MOLLER experiment. It assumes Gaussian process noise, so it requires a simulation where we are well outside of Poisson statistics regime. It currently uses the bayes_opt module, but presumably there are othre options (MC-Stan, even sklearn).

User input to this script, if you wish to adapt it, should be a simulation macro and an analysis macro that returns a target to optimize. 

In [ ]:
# Working directory, assumes build/ will contain remoll and reroot
remoll   = "/home/wdconinc/git/remoll"
# Geometry file to modify with parameters
geomfile = "geometry/pion/Lucite/pionDetectorLucite.gdml"
# Simulation macro, all output to cerr, cout is ignored
macro    = "macros/pion/pionDetectorLucite.mac"
# Analysis macro which produces a floating point number as last line
analysis = 'analysis/pion/pionDetectorLucite_pe.C("pionDetectorLucite_pi.root")'

In [ ]:
import subprocess
import fileinput
import math
import re

def set_geometry(file, name, value):
    """
    Modify the value of the gdml variable <name> to <value> in <file>.
    """
    for line in fileinput.input(file, inplace = True):
        print(re.sub(r'(name="%s"[^\>]*value)="[^"]*"' % name, r'\1="%s"' % str(value), line), end = '')
    fileinput.close()
    

def run_simulation(
    """
    Run a single simulation step with specified parameters.
    """
        N_P = 3,
        pionDetectorLucitePlaneThickness = 1,
        pionDetectorLuciteInsideWidth = 30,
        pionDetectorLuciteOutsideWidth = 30,
        pionDetectorLuciteHeight = 25,
        pionDetectorLuciteTheta = 0,
        pionDetectorLuciteWedgeSide = 1,
        pionDetectorLuciteWedgeAngle = 45,
        pionDetectorLuciteReflectorHeight = 10,
        pionDetectorLuciteReflectorWidth = 40,
        pionDetectorLuciteReflectorDepth = 4,
        pionDetectorLuciteReflectorTheta = 0,
        pionDetectorLuciteLightGuideHeight = 40,
        pionDetectorLuciteLightGuideTheta = 0,
        pionDetectorLucitePMTDiameter = 3):

    # Require integer values for the following parameters
    N_P = int(N_P)
    pionDetectorLucitePlaneThickness = int(pionDetectorLucitePlaneThickness)
    pionDetectorLuciteReflectorDepth = int(pionDetectorLuciteReflectorDepth)
    pionDetectorLucitePMTDiameter    = int(pionDetectorLucitePMTDiameter)

    set_geometry(geomfile, "N_P", N_P)
    set_geometry(geomfile, "pionDetectorLucitePlaneThickness",   pionDetectorLucitePlaneThickness * 2.54)
    set_geometry(geomfile, "pionDetectorLuciteInsideWidth",      pionDetectorLuciteInsideWidth)
    set_geometry(geomfile, "pionDetectorLuciteOutsideWidth",     pionDetectorLuciteOutsideWidth)
    set_geometry(geomfile, "pionDetectorLuciteHeight",           pionDetectorLuciteHeight)
    set_geometry(geomfile, "pionDetectorLuciteTheta",            pionDetectorLuciteTheta)
    set_geometry(geomfile, "pionDetectorLuciteWedgeSide",        pionDetectorLuciteWedgeSide)
    set_geometry(geomfile, "pionDetectorLuciteWedgeAngle",       pionDetectorLuciteWedgeAngle)
    set_geometry(geomfile, "pionDetectorLuciteReflectorHeight",  pionDetectorLuciteReflectorHeight)
    set_geometry(geomfile, "pionDetectorLuciteReflectorWidth",   pionDetectorLuciteReflectorWidth)
    set_geometry(geomfile, "pionDetectorLuciteReflectorDepth",   pionDetectorLuciteReflectorDepth * 2.54)
    set_geometry(geomfile, "pionDetectorLuciteReflectorTheta",   pionDetectorLuciteReflectorTheta)
    set_geometry(geomfile, "pionDetectorLuciteLightGuideHeight", pionDetectorLuciteLightGuideHeight)
    set_geometry(geomfile, "pionDetectorLuciteLightGuideTheta",  pionDetectorLuciteLightGuideTheta)
    set_geometry(geomfile, "pionDetectorLucitePMTDiameter",      pionDetectorLucitePMTDiameter * 2.54)
    
    proc = subprocess.run(["build/remoll", macro],
                   cwd = remoll,
                   stdout = subprocess.DEVNULL,
                   stderr = subprocess.DEVNULL)
    
    proc = subprocess.run(["build/reroot", "-l", "-q", analysis],
                   cwd = remoll,
                   stdout = subprocess.PIPE, 
                   stderr = subprocess.DEVNULL,
                   encoding = 'utf-8')

    # Collect output values: second to last line (last line empty), first word
    number_of_pes = float(proc.stdout.split('\n')[-2].split(' ')[0])
    
    # Detector volume
    area = 0.5 * (pionDetectorLuciteInsideWidth + pionDetectorLuciteOutsideWidth) * pionDetectorLuciteHeight
    volume = N_P * pionDetectorLucitePlaneThickness * area / math.cos(math.radians(pionDetectorLuciteTheta)

    return number_of_pes / volume

In [ ]:
# Bounded region of parameter space, with names of variables
# Order here must correspond with what the target function expects
pbounds = {
    'N_P': (1, 5),
    'PlaneThickness': (1, 5),
    'InsideWidth': (10, 40),
    'OutsideWidth': (10, 40),
    'Height': (5, 40),
    'Theta': (-45, +45),
    'WedgeSide': (-1, 1),
    'WedgeAngle': (0, 60),
    'ReflectorHeight': (0, 20),
    'ReflectorWidth': (20, 50),
    'ReflectorDepth': (10, 25),
    'ReflectorTheta': (-45, +45),
    'LightGuideHeight': (10, 40),
    'LightGuideTheta': (-45, +45),
    'PMTDiameter': (2, 5)
}

In [ ]:
!pip install --user bayesian-optimization

from bayes_opt import BayesianOptimization

# Setup Bayesian optimizer
optimizer = BayesianOptimization(
    f = run_simulation,
    pbounds = pbounds,
    verbose = 2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state = 1,
)

In [ ]:
# Now start optimization from init_points exploration points and n_iter iterations
optimizer.maximize(
    init_points = 2,
    n_iter = 10,
)